In [4]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import gc
import os
# import toad
import re
import catboost as cat
import lightgbm as lgb
from sklearn.metrics import  f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split

In [5]:
os.listdir('./train')

['annual_report_info.csv',
 'base_info.csv',
 'change_info.csv',
 'entprise_info.csv',
 'news_info.csv',
 'other_info.csv',
 'tax_info.csv']

In [6]:
entprise_info = pd.read_csv('./train/entprise_info.csv')
entprise_evaluate = pd.read_csv('entprise_evaluate.csv')
annual_report_info = pd.read_csv('./train/annual_report_info.csv')
change_info = pd.read_csv('./train/change_info.csv')
base_info = pd.read_csv('./train/base_info.csv')
news_info = pd.read_csv('./train/news_info.csv')
other_info = pd.read_csv('./train/other_info.csv')
tax_info = pd.read_csv('./train/tax_info.csv')

In [7]:
test = pd.read_csv('entprise_evaluate.csv')

In [15]:
test.head(10)

,id,score
0,82750f1b9d1223508ee329d47e27d35176c93eb9f35e9c1a,NaN
1,f000950527a6feb670cc1c87c2025f3922aaa4a0206a0a33,NaN
2,e9f7b28ec10e04700ef4db75a494f9a1e8e8b09555e6afa1,NaN
3,beb4aaaa89e0a0ae9d77bd5d7665be6342f552f51840cf19,NaN
4,e9f7b28ec10e0470ee4172cec0133b6826c34f27d3dff204,NaN
5,d8071a739aa75a3ba10ae8205ae3c7b267bdbe32cd6cb582,NaN
6,f000950527a6feb63dcc53f7fc8ccac0eb92d37b34f18854,NaN
7,4a8fc4f3fb4d8a0ff806928b39e9b5e776d9b0d2d32bce0a,NaN
8,f000950527a6feb678e9430f14270a010ec5e747a694829a,NaN
9,d8071a739aa75a3bb4ea6cf0f8ce3b13082101f40a426037,NaN


In [58]:
entprise_evaluate.head(10)

,id,score
0,82750f1b9d1223508ee329d47e27d35176c93eb9f35e9c1a,NaN
1,f000950527a6feb670cc1c87c2025f3922aaa4a0206a0a33,NaN
2,e9f7b28ec10e04700ef4db75a494f9a1e8e8b09555e6afa1,NaN
3,beb4aaaa89e0a0ae9d77bd5d7665be6342f552f51840cf19,NaN
4,e9f7b28ec10e0470ee4172cec0133b6826c34f27d3dff204,NaN
5,d8071a739aa75a3ba10ae8205ae3c7b267bdbe32cd6cb582,NaN
6,f000950527a6feb63dcc53f7fc8ccac0eb92d37b34f18854,NaN
7,4a8fc4f3fb4d8a0ff806928b39e9b5e776d9b0d2d32bce0a,NaN
8,f000950527a6feb678e9430f14270a010ec5e747a694829a,NaN
9,d8071a739aa75a3bb4ea6cf0f8ce3b13082101f40a426037,NaN


In [17]:
annual_report_info.head(10)

,id,ANCHEYEAR,STATE,FUNDAM,MEMNUM,FARNUM,ANNNEWMEMNUM,ANNREDMEMNUM,EMPNUM,EMPNUMSIGN,...,DISPERNUM,UNENUM,COLEMPLNUM,RETEMPLNUM,DISEMPLNUM,UNEEMPLNUM,WEBSITSIGN,FORINVESTSIGN,STOCKTRANSIGN,PUBSTATE
0,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2017.0,2.0,5.0,NaN,NaN,NaN,NaN,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2018.0,2.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
2,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2017.0,2.0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
3,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2018.0,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,...,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
4,9c7fa510616a68309e4badf2a7a3123c0462fb85bf28ef17,2017.0,2.0,5.0,NaN,NaN,NaN,NaN,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
5,9c7fa510616a68309e4badf2a7a3123c0462fb85bf28ef17,2018.0,2.0,22.0,NaN,NaN,NaN,NaN,22.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
6,755db3b5c5f74eb48564a8be9d4a9d7038ed96bc2eea645c,2018.0,2.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
7,e9f7b28ec10e0470287f274dd5a327519e74d2eb9506faad,2016.0,2.0,NaN,NaN,NaN,NaN,NaN,8.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0
8,e9f7b28ec10e0470287f274dd5a327519e74d2eb9506faad,2017.0,2.0,NaN,NaN,NaN,NaN,NaN,8.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0
9,e9f7b28ec10e0470287f274dd5a327519e74d2eb9506faad,2018.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0


In [18]:
change_info.head(10)

,id,bgxmdm,bgq,bgh,bgrq
0,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,939.0,9dec12da51cdb672a91b4a8ae0e0895f7bfeb243dfa3e0c8,9dec12da51cdb672a91b4a8ae0e0895f4a56cbe3deca98...,2.019060e+13
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,112.0,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,2.019060e+13
2,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,111.0,54ca436ffb87f24c820178b45fcc3a7b,f80e3376abcf81ad2a279d6d99046153,2.017013e+13
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,128.0,f1fdb1c866dc96638cbfb8b788b91393,1eca8a0d8beca58d988f7dccab5dc868,2.017013e+13
4,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,925.0,54ca436ffb87f24c820178b45fcc3a7b,f80e3376abcf81ad2a279d6d99046153,2.017013e+13
5,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,120.0,5f8f15bc303b715e4fc0d66e1c15aadf2ce0c00c5c48a9...,5f8f15bc303b715e60804147c4abf9804ac775197ec9ee...,2.018082e+13
6,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,939.0,9dec12da51cdb672a91b4a8ae0e0895f05fd8cfed5ef02...,9dec12da51cdb672a91b4a8ae0e0895f4a56cbe3deca98...,2.019033e+13
7,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,112.0,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,2.019033e+13
8,216bd2aaf4d07924b4a106be25791281e2a6d9e54eaee13b,137.0,696844db768cef6c,2cae8fd090601edc,2.020032e+13
9,216bd2aaf4d07924b4a106be25791281e2a6d9e54eaee13b,137.0,696844db768cef6c,2cae8fd090601edc,2.020032e+13


In [4]:
base_info.head(10)

,id,oplocdistrict,industryphy,industryco,dom,opscope,enttype,enttypeitem,opfrom,opto,...,enttypeminu,midpreindcode,protype,oploc,regcap,reccap,forreccap,forregcap,congro,enttypegb
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,340223,M,7513.0,31487d8f256f16bd6244b7251be2ebb24d1db51663c654...,纳米新材料、机械设备、五金配件加工、销售及技术推广服务，道路货物运输。（依法须经批准的项目，...,1100,1150.0,2019-07-11 00:00:00,NaN,...,1151.0,NaN,NaN,2367b4cac96d8598,50.0,NaN,NaN,NaN,NaN,1151
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,340222,O,8090.0,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,健身服务。（依法须经批准的项目，经相关部门批准后方可开展经营活动）,9600,NaN,2017-09-06,NaN,...,NaN,NaN,NaN,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,10.0,NaN,NaN,NaN,NaN,9600
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,340202,R,9053.0,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,文化娱乐经纪人服务；境内文艺活动组织与策划；文化艺术交流活动组织策划；演出经纪；其他文化艺术...,1100,1150.0,2020-09-14 14:46:30,NaN,...,1151.0,NaN,NaN,2367b4cac96d8598,100.0,NaN,NaN,NaN,NaN,1151
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,340221,L,7212.0,746df9aaed8578571760c563abe882c8ba25209fc6d5db...,投资管理及咨询(证券、期货除外)；企业管理。（依法须经批准的项目，经相关部门批准后方可开展经...,4500,4540.0,2015-09-30,NaN,...,NaN,NaN,NaN,2367b4cac96d8598,10.0,NaN,NaN,NaN,NaN,4540
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,340202,R,8810.0,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,境内文化艺术交流活动策划；企业形象策划；礼仪庆典服务；翻译服务；专利代理；广告设计、制作、代...,1100,1130.0,2017-12-01,2067-11-30,...,NaN,NaN,NaN,2367b4cac96d8598,100.0,NaN,NaN,NaN,NaN,1130
5,da8691b210adb3f67820f5e0c87b337d63112cee52211888,340207,R,9019.0,ca213febe80e171c3b9617e39b49d64e79eaa1699bd4df...,棋牌娱乐服务；理发服务。（依法须经批准的项目，经相关部门批准后方可开展经营活动）,9600,NaN,2019-09-29,NaN,...,NaN,NaN,NaN,31487d8f256f16bd6244b7251be2ebb2918919c75535b4...,10.0,NaN,NaN,NaN,NaN,9600
6,9c7fa510616a6830b878f3c8c4317d93e1b022e7f22ae231,340222,O,8052.0,ca213febe80e171cd550f179e9cf5434d53fa8601fba34...,足疗服务（依法须经批准的项目，经相关部门批准后方可开展经营活动）,9600,NaN,2020-08-03 00:00:00,NaN,...,NaN,NaN,NaN,7c29d90151b88c9f396c6c8374278fe7cab4859047594f...,20.0,NaN,NaN,NaN,NaN,9600
7,9c7fa510616a68309e4badf2a7a3123c0462fb85bf28ef17,340222,O,8111.0,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,汽车维修服务，汽车零配件批发兼零售。（依法须经批准的项目，经相关部门批准后方可开展经营活动）,9600,NaN,2017-07-10,NaN,...,NaN,NaN,NaN,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,15.0,NaN,NaN,NaN,NaN,9600
8,216bd2aaf4d07924b4a106be25791281e2a6d9e54eaee13b,340203,M,7519.0,31487d8f256f16bd6244b7251be2ebb2104d8ea0072d61...,机电设备批发；机电设备零售；机电设备安装及维修；太阳能热水器设备及配件、太阳能光伏设备及配件...,1100,1190.0,2020-03-10 11:52:15,NaN,...,NaN,NaN,NaN,2367b4cac96d8598,300.0,NaN,NaN,NaN,NaN,1190
9,743e550a617316d5772a00182284976e17d42b6f0ca6d374,340208,O,8051.0,31487d8f256f16bd6244b7251be2ebb2918919c75535b4...,足浴服务（依法须经批准的项目，经相关部门批准后方可开展经营活动）。***,9600,NaN,2016-02-29,NaN,...,NaN,NaN,NaN,ca213febe80e171c6293b0274cbe9e27b4b53b5313603c...,10.0,NaN,NaN,NaN,NaN,9600


In [6]:
toad.detect(base_info).iloc[:,:4]

,type,size,missing,unique
id,object,24865,0.00%,24865
oplocdistrict,int64,24865,0.00%,16
industryphy,object,24865,0.00%,20
industryco,float64,24865,0.00%,345
dom,object,24865,0.00%,23278
opscope,object,24865,0.00%,20815
enttype,int64,24865,0.00%,17
enttypeitem,float64,24865,33.03%,31
opfrom,object,24865,0.00%,6620
opto,object,24865,64.51%,5746


In [44]:
base_info.enttypegb.unique()

array([1151, 9600, 4540, 1130, 1190, 1153, 1110, 4533, 1152, 2190, 9100,
       1222, 6150, 2151, 5110, 5150, 1229, 2213, 2221, 4531, 1221, 4560,
       6110, 6180, 6810, 1212, 2153, 2152, 2219, 1211, 3200, 2229, 2130,
       1123, 5810, 2223, 4410, 2211, 3100, 1140, 2212, 5430, 1122, 2121,
       4532, 2123, 5180, 6120, 3500, 4420, 3400, 2110, 1121], dtype=int64)

In [41]:
base_info.enttypeminu.unique()

array([1151.,   nan, 1153., 4533., 1152., 1222., 2151., 1229., 2213.,
       2221., 4531., 1221., 1212., 2153., 2152., 2219., 1211., 2229.,
       1123., 2223., 2211., 2212., 1122., 2121., 4532., 2123., 1121.])

In [8]:
base_info.drop(['ptbusscope','midpreindcode'],axis=1,inplace=True)
# base_info.fillna("unknown",)
def handle_opscope(x):
    x = x.strip('***')
    x = x.strip('（依法须经批准的项目，经相关部门批准后方可开展经营活动）')
    x = x.replace("。","")
    x = x.replace("，","；")
    return len(x.split("；"))
base_info['opscope']=base_info['opscope'].apply(handle_opscope)

# 存在问题，base_info['opto']的缺失值很多
base_info['op_span_year'] = pd.to_datetime(base_info['opto']).dt.year - pd.to_datetime(base_info['opfrom']).dt.year
base_info['op_span_day'] =  pd.to_datetime(base_info['opto']).dt.year - pd.to_datetime(base_info['opfrom']).dt.year

base_info['state_encode'] = base_info.state.map({'4':1,'6':2,'7':3,'11':4,'13':5,'20':6})

# opform特征处理
def handle_opform(x):
    if x == '   ':
        return np.NaN
    elif x == '02-以家庭共有财产作为个人出资':
        return '02'
    return x
base_info['opform']=base_info['opform'].apply(handle_opform)

# count encoding
for fea in ['oplocdistrict', 'industryphy', 'dom','enttype','enttypeitem','state','orgid','jobid',\
            'regtype','compform','opform','venind','enttypeminu','protype','oploc','enttypegb',]:
    base_info[fea+'_counts'] = base_info.groupby([fea])['id'].transform('count')
    
# base_info.fillna('unknown', inplace=True)
base_info.fillna("", inplace=True)

In [10]:
base_info.head()

,id,oplocdistrict,industryphy,industryco,dom,opscope,enttype,enttypeitem,opfrom,opto,...,orgid_counts,jobid_counts,regtype_counts,compform_counts,opform_counts,venind_counts,enttypeminu_counts,protype_counts,oploc_counts,enttypegb_counts
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,340223,M,7513,31487d8f256f16bd6244b7251be2ebb24d1db51663c654...,2,1100,1150,2019-07-11 00:00:00,,...,929,163,23792,,,,4795,,18792,4795
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,340222,O,8090,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,1,9600,,2017-09-06,,...,435,202,23792,10610,8186,8115,,,337,8193
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,340202,R,9053,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,17,1100,1150,2020-09-14 14:46:30,,...,3617,260,23792,,,,4795,,18792,4795
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,340221,L,7212,746df9aaed8578571760c563abe882c8ba25209fc6d5db...,2,4500,4540,2015-09-30,,...,1361,87,23792,,575,,,,18792,757
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,340202,R,8810,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,6,1100,1130,2017-12-01,2067-11-30,...,2261,53,23792,,,,,,18792,7467


In [11]:
train = pd.merge(entprise_info,base_info,on='id',how='left')
test = pd.merge(entprise_evaluate,base_info,on='id',how='left')
train.drop(['id'],axis=1,inplace=True)
test.drop(['id','score'],axis=1,inplace=True)
print(train.shape,test.shape)

(14865, 50) (10000, 49)


In [12]:
y = train.pop('label')

In [13]:
# cat_indexes = set()

except_cat_feas = ['empnum','parnum','exenum','regcap','reccap','forreccap','forregcap','congro'] + \
                  [col for col in train.columns.tolist() if col.endswith('counts')]
cat_feas = list(set(train.columns.tolist()) - set(except_cat_feas))
cat_index = [list(train.columns).index(fea) for fea in cat_feas]
train.iloc[:,cat_index] = train.iloc[:,cat_index].astype(str)
test.iloc[:,cat_index] = test.iloc[:,cat_index].astype(str)
print(cat_index)

[29, 20, 16, 1, 7, 11, 9, 31, 6, 23, 19, 8, 30, 5, 2, 4, 22, 3, 0, 14, 21, 10, 32, 13, 12]


In [14]:
train

,oplocdistrict,industryphy,industryco,dom,opscope,enttype,enttypeitem,opfrom,opto,state,...,orgid_counts,jobid_counts,regtype_counts,compform_counts,opform_counts,venind_counts,enttypeminu_counts,protype_counts,oploc_counts,enttypegb_counts
0,340202,J,6760.0,ca213febe80e171c5ab458cd1d4bd374e6d04e8fbdfe83...,3,4500,4530.0,2016-12-07,2046-12-06,6,...,3617,1177,23792,,,,1371,,18792,1371
1,340207,O,8060.0,ca213febe80e171c3b9617e39b49d64ee24f8f59950c35...,1,9600,,2019-04-08,,6,...,199,156,23792,10610,8186,8115,,,1,8193
2,340208,M,7590.0,31487d8f256f16bd6244b7251be2ebb2918919c75535b4...,11,1100,1130.0,2020-07-10 15:12:30,,6,...,453,45,23792,,,,,,18792,7467
3,340202,M,7514.0,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,2,1100,1150.0,2019-06-21 00:00:00,,6,...,2261,831,23792,,,,4795,,18792,4795
4,340225,O,8111.0,31487d8f256f16bd6244b7251be2ebb227ca54d864983f...,1,9600,,2020-04-07 00:00:00,,6,...,37,11,23792,10610,8186,8115,,,1,8193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14860,340203,M,7511.0,31487d8f256f16bd6244b7251be2ebb2104d8ea0072d61...,15,1100,1150.0,2020-07-14 14:02:08,,6,...,2098,307,23792,,,,4795,,18792,4795
14861,340221,O,8219.0,746df9aaed85785706df2df96d3addcbb7062acc711f5a...,7,1100,1150.0,2019-12-19 00:00:00,2049-12-16 00:00:00,6,...,1361,134,23792,,,,4795,,18792,4795
14862,340222,O,8040.0,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,2,9600,,2019-12-09 00:00:00,,6,...,435,224,23792,10610,8186,8115,,,337,8193
14863,340202,Q,8499.0,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,1,1100,1130.0,2018-01-18 00:00:00,,7,...,2261,831,23792,,,,,,18792,7467


In [37]:
x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.1, random_state=10)
cat_params = {               
         'learning_rate': 0.1,
         'depth': 8,
         'l2_leaf_reg': 10,
         'random_seed': 10,
         'cat_features':cat_index,
         'task_type':'GPU'
         }
# cat_model= cat.CatBoostClassifier(iterations=20000,**cat_params)
cat_model= cat.CatBoostRegressor(iterations=20000,**cat_params)
cat_model.fit(x_train, y_train,eval_set=(x_test, y_test),use_best_model=True,verbose=100,early_stopping_rounds=300)
y_pred = cat_model.predict(x_test)
y_pred_bin = np.array([1 if pred>=0.5 else 0 for pred in y_pred ])
print('The f1-score is:', f1_score(y_test, y_pred_bin))
print('The precision_score is:', precision_score(y_test, y_pred_bin))
print('The recall_score is:', recall_score(y_test, y_pred_bin))
feas_importances_dict = {col:importance for col,importance in zip(list(train.columns),list(cat_model.feature_importances_))}
print('Cat Feature importances:\n',dict(sorted(feas_importances_dict.items(),key=lambda item:item[1], reverse=True)))

# cat_test = cat_model.predict_proba(test)
cat_test = cat_model.predict(test)

0:	learn: 0.2347678	test: 0.2261573	best: 0.2261573 (0)	total: 78ms	remaining: 25m 59s
100:	learn: 0.1356606	test: 0.1430369	best: 0.1430329 (98)	total: 9.1s	remaining: 29m 52s
200:	learn: 0.1292099	test: 0.1368032	best: 0.1368032 (200)	total: 18.2s	remaining: 29m 57s
300:	learn: 0.1266758	test: 0.1354814	best: 0.1353688 (294)	total: 26.4s	remaining: 28m 45s
400:	learn: 0.1248723	test: 0.1350854	best: 0.1350712 (378)	total: 34.2s	remaining: 27m 51s
500:	learn: 0.1242132	test: 0.1348924	best: 0.1348000 (481)	total: 41.6s	remaining: 26m 58s
600:	learn: 0.1235432	test: 0.1350002	best: 0.1348000 (481)	total: 49.4s	remaining: 26m 34s
700:	learn: 0.1229944	test: 0.1352274	best: 0.1348000 (481)	total: 56.9s	remaining: 26m 7s
bestTest = 0.1347999801
bestIteration = 481
Shrink model to first 482 iterations.
The f1-score is: 0.8111111111111112
The precision_score is: 0.8202247191011236
The recall_score is: 0.8021978021978022
Cat Feature importances:
 {'industryphy': 61.849320168097826, 'industry

In [41]:
cat_test = [0 if i<0 else 1 if i>1 else i  for i in cat_test]
# print(cat_test[:,1]>0.5)
# entprise_evaluate['score'] = cat_test[:,1]
entprise_evaluate['score'] = cat_test
result = entprise_evaluate.to_csv('11_10_0.811.csv',index=False)
print('OK')

OK


In [20]:
news_info.head(10)

,id,positive_negtive,public_date
0,f000950527a6feb62669d6a175fe6fdccd1eb4f7ca8e5016,积极,2016-12-30
1,f000950527a6feb6e8bd9919e2ca363359bcfa997a0f9de7,中立,2017-08-09
2,f000950527a6feb6e8bd9919e2ca363359bcfa997a0f9de7,消极,2016-02-29
3,d8071a739aa75a3bcf6fb0041ee883243251d30025ab9d45,中立,2018-06-08
4,f000950527a6feb6d71de3382afa0bc5ff87bb65477f698a,积极,2015-06-29
5,f000950527a6feb6d71de3382afa0bc5ff87bb65477f698a,积极,2015-06-15
6,f000950527a6feb65929509d9be855bf75b7337d4465843e,积极,2019-10-26
7,f000950527a6feb65929509d9be855bf75b7337d4465843e,积极,2017-11-01
8,f000950527a6feb68b2de1c0fb54cf2919d4029742a7a1c4,中立,2018-04-20
9,f000950527a6feb68b2de1c0fb54cf2919d4029742a7a1c4,中立,2018-01-08


In [21]:
other_info.head(10)

,id,legal_judgment_num,brand_num,patent_num
0,f000950527a6feb6d340f91da09e61347d8200cd2f0d1602,4.0,NaN,NaN
1,f000950527a6feb608dd9322b74a99f60851207f36a3c94c,1.0,NaN,NaN
2,d8071a739aa75a3b9f23966f8dae78fd226c272515b9c255,2.0,NaN,NaN
3,216bd2aaf4d079242209b1496f81a36c7abed9dd0bb65ed3,NaN,1.0,NaN
4,e9f7b28ec10e0470de9631c789f49acdd4e7cf9ed6db094b,NaN,2.0,NaN
5,d8071a739aa75a3bcf6fb0041ee883243251d30025ab9d45,1.0,NaN,NaN
6,f000950527a6feb6f97af739bb95531db891a11df80bdb8b,3.0,NaN,NaN
7,f000950527a6feb65929509d9be855bf75b7337d4465843e,47.0,NaN,NaN
8,f000950527a6feb6a81704c38a21ae17f4f09eaa9aa77bea,NaN,3.0,NaN
9,f000950527a6feb6c0ab37e6eb4a4bf25ee20f0ff9ff070d,NaN,1.0,NaN


In [22]:
tax_info.head(10)

,id,START_DATE,END_DATE,TAX_CATEGORIES,TAX_ITEMS,TAXATION_BASIS,TAX_RATE,DEDUCTION,TAX_AMOUNT
0,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,工伤保险（单位）,72530.75,0.0003,-0.04,21.80
1,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,失业保险（单位）,72530.75,0.0003,-0.04,21.80
2,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,医疗保险（单位）,72530.75,0.0003,-0.04,21.80
3,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,企业养老保险基金（单位）,72530.75,0.0003,-0.04,21.80
4,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,烟叶收购,72530.75,0.0003,-0.04,21.80
5,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,原油,72530.75,0.0003,-0.04,21.80
6,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,技术合同,72530.75,0.0003,-0.04,21.80
7,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,城市维护建设税,金融保险营业税额,2112.56,0.0700,0.00,147.88
8,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,城市维护建设税,优惠税率１,2112.56,0.0700,0.00,147.88
9,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,城市维护建设税,铁路运输,2112.56,0.0700,0.00,147.88
